<a href="https://colab.research.google.com/github/GehrmanSparrow116/DJS-CODE-AI/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:10%]")
train_texts = dataset['text'][:10000]
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # set EOS as padding
tokenized_texts = [tokenizer(t, truncation=True, padding="max_length", max_length=64)
                   for t in train_texts]
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_texts):
        self.input_ids = [torch.tensor(t['input_ids']) for t in tokenized_texts]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {'input_ids': self.input_ids[idx], 'labels': self.input_ids[idx]}

dataset = TextDataset(tokenized_texts)


from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("distilgpt2")


from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./mini_gpt",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    fp16=True,
    report_to=[]  # disables all logging backends (wandb, tensorboard)
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



trainer.train()


while True:
    prompt = input("Enter prompt (or type 'exit' to quit): ")
    if prompt.lower() == "exit":
        break
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )
    print("\nGenerated Text:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))

Step,Training Loss
100,2.472900
200,1.774000
300,1.618700
400,1.658900
500,1.682900
600,1.607500
700,1.529400
800,1.557100
900,1.679700
1000,1.419300


Enter prompt (or type 'exit' to quit): warrior


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Text:
 warrior – chief – chief – officer of the Archaeological Survey of India ( Archaeological Survey ) , Kurthasar Gupta of Karnataka , claimed that the archaeological record indicates that the archaeologist 's position in this period was ' the same , but that the archaeologist had a different opinion regarding the occurrence of archaeology in Karnataka . ' He claimed that the archaeological record indicates that the archaeologist 's position in this period was ' the same , but that the archaeologist 's position
Enter prompt (or type 'exit' to quit): who am i


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Text:
 who am i the sole god , that is the divine creation of the whole world , is the creation of the whole world . 

Enter prompt (or type 'exit' to quit): exit
